In [ ]:
from getpass import getpass
import pandas as pd
from utils import load_encrypted_xlsx
import numpy as np

In [ ]:
registry_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/dci_sah/data/sos_sah_data/post_hoc_modified_aSAH_DATA_2009_2023_24122023.xlsx'
hta_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/dci_sah/data/pdms_data/Transfer Urs.pietsch@kssg.ch 22.01.24, 15_34/20240116_SAH_SOS_Blutdruecke.csv'
registry_pdms_correspondence_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/dci_sah/data/pdms_data/registry_pdms_correspondence.csv'

In [ ]:
registry_df = load_encrypted_xlsx(registry_data_path)
hta_df = pd.read_csv(hta_data_path, sep=';', decimal='.')
registry_pdms_correspondence_df = pd.read_csv(registry_pdms_correspondence_path)

In [ ]:
registry_pdms_correspondence_df.rename(columns={'JoinedName': 'Name'}, inplace=True)
registry_pdms_correspondence_df['Date_birth'] = pd.to_datetime(registry_pdms_correspondence_df['Date_birth'],
                                                                   format='%Y-%m-%d')
hta_df = hta_df.merge(registry_pdms_correspondence_df, on='pNr', how='left')
registry_df = registry_df.merge(registry_pdms_correspondence_df, on=['SOS-CENTER-YEAR-NO.', 'Name', 'Date_birth'],
                                how='left')

In [ ]:
dci_df = registry_df[['pNr', 'DCI_ischemia', 'Date_admission']]
hta_df = hta_df.merge(dci_df, how='left', left_on='pNr', right_on='pNr')

In [ ]:
hta_df['timeBd'] = pd.to_datetime(hta_df['timeBd'], format='%Y-%m-%d %H:%M:%S.%f')
hta_df['relative_time'] = hta_df.apply(lambda x: (x['timeBd'] - x['Date_admission']).total_seconds() / 3600, axis=1)
hta_df['day'] = hta_df['relative_time'].apply(lambda x: int(x / 24) if pd.notnull(x) else np.nan)

In [ ]:
# for every pnr get max for column day
hta_df['max_day'] = hta_df.groupby('pNr')['day'].transform('max')

In [ ]:
# find patients who do not have at least one value per day
hta_df['n_day']= hta_df.groupby('pNr')['day'].transform('nunique')

In [ ]:
hta_df['missing_days'] = hta_df['max_day'] + 1 - hta_df['n_day']

In [ ]:
missing_df = hta_df[['pNr', 'max_day', 'n_day', 'missing_days']].drop_duplicates()

In [ ]:
missing_df.missing_days.value_counts()

In [ ]:
missing_pnrs = missing_df[missing_df.missing_days > 0].pNr.unique()

In [ ]:
registry_df[registry_df.pNr.isin(missing_pnrs)][['pNr', 'Name', 'Date_birth', 'DCI_ischemia', 'Date_admission']]